    16_230110_114_agg_combine_map1
        
        data : /aiffel/000_aiffelthon_230111/3김연수/002_data_dir/001_map_data/kyungki_region.json
        /aiffel/aiffel/000_aiffelthon_230111/3김연수/002_data_dir/004_node_data/20210823_socar_zone.csv
        /aiffel/000_aiffelthon_230111/3김연수/002_data_dir/003_search_data/happy_house.csv
        /aiffel/000_aiffelthon_230111/3김연수/002_data_dir/003_search_data/k_university.csv

In [162]:
#1. package load
import requests
import json
import os
import math
import time
import numpy as np
import pandas as pd
import geopandas as gpd
import pyproj
import folium
import matplotlib.pyplot as plt
import seaborn as sns

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap

import warnings
warnings.filterwarnings(action='ignore') 

print(pd.__version__)
print(gpd.__version__)
print(folium.__version__)

1.3.3
0.10.1
0.12.1


In [163]:
#2. 프로젝트(1)  ... data load

path = '/aiffel/aiffel/000_aiffelthon_230111/3김연수/002_data_dir/004_node_data/20210823_socar_zone.csv'

socar_zone_info_df = pd.read_csv(path)
socar_zone_info_df.head()

,zone_name,lng,lat,region1,region2,region3
0,비전2 파출소사거리,127.102939,36.995989,경기도,평택시,비전동
1,수원시청역 9번출구,127.032953,37.262378,경기도,수원시 팔달구,인계동
2,목포 상동현대아파트 앞,126.415655,34.811289,전라남도,목포시,상동
3,단국대 산학협력관 앞,127.166626,36.832965,충청남도,천안시 동남구,신부동
4,목원대학교 앞,127.337858,36.329906,대전광역시,서구,도안동


In [164]:
k_socar_zone_df = socar_zone_info_df[socar_zone_info_df['region1'] == '경기도']

In [165]:
#3.  행복주택 load
path = os.getenv('HOME') + '/aiffel/000_aiffelthon_230111/3김연수/002_data_dir/003_search_data/happy_house.csv'
happy_house_df = pd.read_csv(path)
happy_house_df.head()

,지역,단지,세대수,주소,위도,경도,x,y,lng,lat
0,고양시 덕양구,삼송11단지,832,경기도 고양시 덕양구 원흥1로 25,37.650992,126.865491,944033.138640,1.961510e+06,126.865540,37.651375
1,고양시 덕양구,고양지축 A-1블록 신혼희망타운 행복주택,250,경기도 고양시 덕양구 오부자로 99,37.653331,126.921037,948927.188681,1.961564e+06,126.921014,37.652148
2,고양시 덕양구,고양지축 A-4BL 행복주택,473,경기도 고양시 덕양구 오부자로 43,37.654038,126.914946,948402.245222,1.961752e+06,126.915050,37.653817
3,고양시 덕양구,고양삼송 LH원흥마을13단지,947,경기도 고양시 덕양구 원흥1로 35,37.652631,126.868888,944239.664819,1.961600e+06,126.867874,37.652197
4,고양시 덕양구,서정마을LH12단지,276,경기도 고양시 덕양구 서정마을2로 17,37.621215,126.847648,942442.993027,1.958197e+06,126.847776,37.621419


In [166]:
happy_house_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지역      95 non-null     object 
 1   단지      95 non-null     object 
 2   세대수     95 non-null     int64  
 3   주소      95 non-null     object 
 4   위도      95 non-null     float64
 5   경도      95 non-null     float64
 6   x       92 non-null     float64
 7   y       92 non-null     float64
 8   lng     92 non-null     float64
 9   lat     92 non-null     float64
dtypes: float64(6), int64(1), object(3)
memory usage: 7.5+ KB


In [167]:
happy_house_df[happy_house_df['lat'].isnull()]

,지역,단지,세대수,주소,위도,경도,x,y,lng,lat
20,남양주시,남양주금곡 행복주택,352,경기도 남양주시 경춘로 899,37.633148,127.201262,NaN,NaN,NaN,NaN
62,파주시,노을빛마을 16단지,320,경기도 파주시 노을빛로 41,37.726922,126.706746,NaN,NaN,NaN,NaN
71,포천시,non,0,non,0.000000,0.000000,NaN,NaN,NaN,NaN


In [168]:
happy_house_df = happy_house_df.dropna(axis=0)

In [169]:
happy_house_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92 entries, 0 to 94
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지역      92 non-null     object 
 1   단지      92 non-null     object 
 2   세대수     92 non-null     int64  
 3   주소      92 non-null     object 
 4   위도      92 non-null     float64
 5   경도      92 non-null     float64
 6   x       92 non-null     float64
 7   y       92 non-null     float64
 8   lng     92 non-null     float64
 9   lat     92 non-null     float64
dtypes: float64(6), int64(1), object(3)
memory usage: 7.9+ KB


In [170]:
#3.  대학교 load
path = os.getenv('HOME') + '/aiffel/000_aiffelthon_230111/3김연수/002_data_dir/003_search_data/k_university.csv'
k_university_df = pd.read_csv(path)
k_university_df.head()

,학교구분,학교코드,학교명,본분교,학제,지역,설립구분,관련법령,법인명,학교상태,...,우편번호,학교개교일,학교홈페이지,총장(대학원장)명,학교대표\r\n번호,학교대표\r\n팩스번호,x,y,lng,lat
0,대학,37.0,한경대학교,본교,대학교,경기,국립,고등교육법,해당없음,기존,...,17579,2012-03-01,www.hknu.ac.kr,이원희,031-670-5114,031-670-5469,978801.712957,1.890391e+06,127.261716,37.011780
1,대학,46.0,가톨릭대학교,본교,대학교,경기,사립,고등교육법,가톨릭학원,기존,...,14662,1995-03-01,www.catholic.ac.kr,원종철,02-2164-4114,02-2164-4754,938317.711386,1.943204e+06,126.802293,37.486022
2,대학,51.0,강남대학교,본교,대학교,경기,사립,고등교육법,강남학원,기존,...,16979,1948-08-12,www.kangnam.ac.kr,윤신일,031-280-3500,031-280-3173,967379.084547,1.919514e+06,127.132051,37.273974
3,대학,56.0,경기대학교,본교,대학교,경기,사립,고등교육법,경기학원,기존,...,16227,1947-11-08,www.kyonggi.ac.kr,전준철(직무대행),031-249-9114,031-243-2194,958688.169183,1.922509e+06,127.033857,37.300618
4,대학,63.0,가천대학교,본교,대학교,경기,사립,고등교육법,가천학원,기존,...,13120,2012-03-01,www.gachon.ac.kr,이길여,031-750-5114,031-753-8828,967163.380457,1.939303e+06,127.128740,37.452335


In [171]:
k_university_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   학교구분         248 non-null    object 
 1   학교코드         248 non-null    float64
 2   학교명          248 non-null    object 
 3   본분교          248 non-null    object 
 4   학제           248 non-null    object 
 5   지역           248 non-null    object 
 6   설립구분         248 non-null    object 
 7   관련법령         248 non-null    object 
 8   법인명          248 non-null    object 
 9   학교상태         248 non-null    object 
 10  중문학교명        247 non-null    object 
 11  영문학교명        247 non-null    object 
 12  addr         249 non-null    object 
 13  주소           248 non-null    object 
 14  Unnamed: 14  248 non-null    object 
 15  Unnamed: 15  248 non-null    object 
 16  Unnamed: 16  248 non-null    object 
 17  영문주소         247 non-null    object 
 18  중문주소         247 non-null    object 
 19  우편번호    

In [172]:
k_university_df[k_university_df['lat'].isnull()]

,학교구분,학교코드,학교명,본분교,학제,지역,설립구분,관련법령,법인명,학교상태,...,우편번호,학교개교일,학교홈페이지,총장(대학원장)명,학교대표\r\n번호,학교대표\r\n팩스번호,x,y,lng,lat
71,대학원대학,672.0,실천신학대학원대학교,본교,전문대학원,경기,사립,고등교육법,실천신학대학원,기존,...,17304,2005-03-01,www.gspt.ac.kr,이정익,031-638-8657,031-638-8659,NaN,NaN,NaN,NaN
184,전문대학,2641.0,경복대학교,본교,전문대학,경기,사립,고등교육법,학교법인 경복대학교,기존,...,12051,1991-11-30,www.kbu.ac.kr,전지용,031-570-9901,031-570-9929,NaN,NaN,NaN,NaN
208,대학원,2859.0,예원예술대학교 문화예술대학원,제2캠퍼스,특수대학원,경기,사립,고등교육법,예원예술대학교,기존,...,11429,2014-03-01,www.culture.yewon.ac.kr,차종순,031-869-0526,031-859-8114,NaN,NaN,NaN,NaN
248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
k_university_df = k_university_df.dropna(axis=0)

In [174]:
k_university_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239 entries, 0 to 246
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   학교구분         239 non-null    object 
 1   학교코드         239 non-null    float64
 2   학교명          239 non-null    object 
 3   본분교          239 non-null    object 
 4   학제           239 non-null    object 
 5   지역           239 non-null    object 
 6   설립구분         239 non-null    object 
 7   관련법령         239 non-null    object 
 8   법인명          239 non-null    object 
 9   학교상태         239 non-null    object 
 10  중문학교명        239 non-null    object 
 11  영문학교명        239 non-null    object 
 12  addr         239 non-null    object 
 13  주소           239 non-null    object 
 14  Unnamed: 14  239 non-null    object 
 15  Unnamed: 15  239 non-null    object 
 16  Unnamed: 16  239 non-null    object 
 17  영문주소         239 non-null    object 
 18  중문주소         239 non-null    object 
 19  우편번호    

In [175]:
# #2. 프로젝트(1)  # MarkerCluster 객체 
# marker_cluster = MarkerCluster().add_to(map)

# #maker 객체 추가
# for idx, row in socar_zone_info_df.iterrows():
#     try:
#         folium.Marker(
#             location=[socar_zone_info_df.loc[idx, "lat"], socar_zone_info_df.loc[idx, "lng"]],
#             popup=socar_zone_info_df.loc[idx, "zone_name"],
#             icon=folium.Icon(color="cadetblue", icon="car", prefix="fa")            
#         ).add_to(marker_cluster)
#     except:
#         pass

In [176]:
#4. 프로젝트(2) ... 3)시각화  ... # Folium 초기 Map 객체 생성 

center = [37.541, 126.986]

map = folium.Map(
    location=center,
    zoom_start= 10
)


In [177]:
# #4. 프로젝트(3)  .. # 2)레이어 적용을 위한 그룹 만들기
# # 대표 그룹 만들기
fg = folium.FeatureGroup(name="groups")
map.add_child(fg)

# 행복주택 그룹 만들기
happy = plugins.FeatureGroupSubGroup(fg, "happy house")
map.add_child(happy)

# 대학교 그룹 만들기
university = plugins.FeatureGroupSubGroup(fg, "university")
map.add_child(university)

# 쏘카 그룹 만들기
socar = plugins.FeatureGroupSubGroup(fg, "socar_zone")
map.add_child(socar)

# 레이어 콘트롤 박스 만들기
folium.LayerControl(collapsed=False).add_to(map)

In [178]:
mg1 = MarkerCluster()
for _, row in k_socar_zone_df.iterrows():
    mg1.add_child(    
        folium.Marker(location = [row['lat'], row['lng']],
               popup=row['zone_name'],
               icon=folium.Icon(color='blue',icon='star')
              )
    )

mg1.add_to(socar)

In [179]:
mg2 = MarkerCluster()
for _, row in happy_house_df.iterrows():
    mg2.add_child(    
        folium.Marker(location = [row['lat'], row['lng']],
               popup=row['단지'],
               icon=folium.Icon(color='red',icon='home')
              )
    )
    
mg2.add_to(happy)

In [180]:
mg3 = MarkerCluster()
for _, row in k_university_df.iterrows():
    mg3.add_child(    
        folium.Marker(location = [row['lat'], row['lng']],
               popup=row['학교명'],
               icon=folium.Icon(color='green',icon='flag')
              )
    )

mg3.add_to(university)

In [181]:
#4 경기 구 data
k_geo_data_path = os.getenv('HOME') + '/aiffel/000_aiffelthon_230111/3김연수/002_data_dir/001_map_data/kyungki_region.json'
k_geo_data = json.load(
    open(k_geo_data_path, encoding="utf-8"))



In [182]:
# #4. 프로젝트(2) ... 3)시각화  ... # Folium 초기 Map 객체 생성 

# center = [37.541, 126.986]

# map = folium.Map(
#     location=center,
#     zoom_start= 10
# )

# folium.GeoJson(
#     k_geo_data,
#     name='경기도'
# ).add_to(map)

# print('슝~')

In [183]:
#4. 프로젝트(3)  ..6) 생성한 map 확인
map
map.add_child(mg1)
map.add_child(mg2)
map.add_child(mg3)


In [184]:
map.save("/aiffel/aiffel/000_aiffelthon_230111/3김연수/003_map_xml/combine_map_2.html")